In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

# ID 리스트
ids = [10092440, 10092030, 10091668, 10090957, 10089974, 10089287, 10088697, 10088294, 10087423, 10086584]

# 크롬 설정
options = webdriver.ChromeOptions()
# options.add_argument('--headless')  # 눈으로 확인하려면 주석처리
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 결과 임시 저장 리스트
results = []

for ntt_id in ids:
    # URL 구성
    url = f'https://www.bok.or.kr/portal/bbs/B0000169/view.do?nttId={ntt_id}&searchCnd=1&searchKwd=&depth2=201213&depth3=201295&pageUnit=12&depth=201295&pageIndex=1&menuNo=201295&oldMenuNo=201295&programType=multiCont'
    
    driver.get(url)

    try:
        # 버튼 클릭 대기 및 클릭
        button_xpath = '//*[@id="board"]/div/div[3]/div/div[3]/button'
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, button_xpath)))
        driver.find_element(By.XPATH, button_xpath).click()
        time.sleep(1)

        # 본문 텍스트 추출
        content_xpath = '//*[@id="board"]/div/div[3]/div/div[3]/div'
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, content_xpath)))
        text = driver.find_element(By.XPATH, content_xpath).text.strip()

        # 날짜 추출
        date_xpath = '//*[@id="board"]/div/div[3]/div/div[4]/div/p[1]/span'
        date = driver.find_element(By.XPATH, date_xpath).text.strip()

        # 결과 저장
        results.append({
            'nttId': ntt_id,
            'Date': date,
            'Content': text
        })

    except Exception as e:
        print(f"❌ {ntt_id} 처리 중 오류 발생:", e)
        results.append({
            'nttId': ntt_id,
            'Date': None,
            'Content': '에러'
        })

# DataFrame 생성
df = pd.DataFrame(results)

# 출력 확인
print(df.head())

# 종료
driver.quit()

❌ 10088697 처리 중 오류 발생: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="board"]/div/div[3]/div/div[4]/div/p[1]/span"}
  (Session info: chrome=138.0.7204.158); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0x541a33+62339]
	GetHandleVerifier [0x0x541a74+62404]
	(No symbol) [0x0x382123]
	(No symbol) [0x0x3ca86e]
	(No symbol) [0x0x3cac0b]
	(No symbol) [0x0x412f72]
	(No symbol) [0x0x3ef404]
	(No symbol) [0x0x4107a3]
	(No symbol) [0x0x3ef1b6]
	(No symbol) [0x0x3be7a2]
	(No symbol) [0x0x3bf644]
	GetHandleVerifier [0x0x7b65c3+2637587]
	GetHandleVerifier [0x0x7b19ca+2618138]
	GetHandleVerifier [0x0x5684aa+220666]
	GetHandleVerifier [0x0x5588d8+156200]
	GetHandleVerifier [0x0x55f06d+182717]
	GetHandleVerifier [0x0x549978+94920]
	GetHandleVerifier [0x0x549b02+95314]
	GetHandleVerifier [0x0x534c4a+9626]
	BaseThreadInitThunk [

In [14]:
df = df.dropna()

In [25]:
df['Date'] = pd.to_datetime(df['Date'].str.replace(r'\D', '', regex=True).str[:5], format='%Y%m').astype(str).str[:7]

C:\Users\campus4D035\AppData\Local\Temp\ipykernel_25472\305888867.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'] = pd.to_datetime(df['Date'].str.replace(r'\D', '', regex=True).str[:5], format='%Y%m').astype(str).str[:7]


In [33]:
df = df.sort_values('Date')[2:][['Date','Content']]


In [35]:
df.to_csv('기자간담회.csv')

In [5]:
import pandas as pd
df = pd.read_csv('기자간담회.csv')
df = df.sort_values('Date')[['Date','Content']]
df = df.dropna()
df

,Date,Content
0,2025-01,공 보 관 - 지금부터 2025년 1월 통화정책방향에 대한 이창용 한국은행 총재의 ...
1,2025-02,공 보 관 - 지금부터 2025년 2월 통화정책방향에 대한 이창용 한국은행 총재의 ...
2,2025-04,공 보 관 - 지금부터 2025년 4월 통화정책방향에 대한 이창용 한국은행 총재의 ...
3,2025-05,공 보 관 - 지금부터 2025년 5월 통화정책방향에 대한 이창용 한국은행 총재의 ...
5,2025-07,공 보 관 - 지금부터 2025년 7월 통화정책방향에 대한 이창용 한국은행 총재의 ...
